Required Improvements:

1. rename sql agent
2. update reorder predictions to reorder threshold as the base/0 value
2. allow more tables to view (check if multiple join statements are working and order by and groupby )
3. explain strategies properly
4. make sure add, remove and edit strategies work for reorder
5. consistent tables
6. allow user to update predictions of oos model and demand forecasting


In [1]:
from agents.reorder_agent import create_reorder_agent 
from agents.sql_agent import create_sql_agent
from langchain_core.messages import HumanMessage

reorder_agent = create_reorder_agent()
sql_agent = create_sql_agent()



/Users/david/Desktop/Learnings/retailDemo/retail_agentic/agents/sql_agent.py:88: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  query_tool = QuerySQLDataBaseTool(db=db)


In [ ]:
#customer related questions
questions = [
    "who are my top customers based on purchase history?",
    "which segment and cluster_explanation does edward baker belong to?",
    "what are the most bought items from customers that belong to cluster_explanation Premium Loyalist?"
    "what are the products pending approval for reordering along with their quantity, in a table format?"
    
    ]

state = {"messages": []}

    #while True:
for question in questions:
        #user_input = input("\nUser > ")
        user_input = question
        print('User Question: ', user_input)
        if user_input.lower() in ["quit", "exit"]: break
        
        state["messages"].append(HumanMessage(content=user_input))
        
        final_state = sql_agent.invoke(state)
        
        final_response_message = final_state['messages'][-1]
        print(f"\nAgent > {final_response_message.content}")

        state['messages'].extend(final_state['messages'])

User Question:  what are the products pending approval for reordering along with their quantity?
---NODE: GENERATE SQL---
 -> Generated SQL: SELECT T1.product_name, T2.reorder_qty FROM products AS T1 INNER JOIN reorders AS T2 ON T1.sku_id = T2.sku_id WHERE T2.status = 'pending'
---NODE: EXECUTE SQL---
Full query string received: SELECT T1.product_name, T2.reorder_qty FROM products AS T1 INNER JOIN reorders AS T2 ON T1.sku_id = T2.sku_id WHERE T2.status = 'pending'
 -> Executing statement 1/1: SELECT T1.product_name, T2.reorder_qty FROM products AS T1 INNER JOIN reorders AS T2 ON T1.sku_id = T2.sku_id WHERE T2.status = 'pending'
 -> DB Results: [('Security Middle', 106), ('Base Here', 68), ('Language Control', 134), ('Cover Letter', 39), ('Others Commercial', 133), ('Notice Arm', 32), ('Cover Letter', 186), ('Decade Election', 158), ('Police Energy', 23), ('House Million', 43), ('Least Especially', 80), ('Manage Foreign', 144), ('Follow Long', 62), ('Choice Environmental', 93), ('Often 

In [ ]:
#reorder questions final
questions = []
questions = [
    "Hey I have a budget of $5000, can you help me stock up my seasonal items",
    

print("Welcome to the Inventory Reordering Supervisor Agent!")
state = {"messages": []}

    #while True:
for question in questions:
        #user_input = input("\nUser > ")
        user_input = question
        print('User Question: ', user_input)
        if user_input.lower() in ["quit", "exit"]: break
        
        state["messages"].append(HumanMessage(content=user_input))
        
        final_state = reorder_agent.invoke(state)
        
        final_response_message = final_state['messages'][-1]
        print(f"\nAgent > {final_response_message.content}")

        state['messages'].extend(final_state['messages'])

In [2]:
#reorder questions
questions = []
questions = [
    "Can you generate a purchase order for me? Budget is $1400.",
    "Hmm, what are the settings for Peak Season Prep?",
    "Okay, for 'Peak Season Prep', change the 'profit' priority to 5",
    "Great, now generate a recommendation with the Peak Season Prep strategy",
    "Okay, let's make some changes. Please update the quantity for SKU0027 to 20 units. Also, remove SKU0067 from the order",
    "Update the max capacity of SKU0020 to 150.",
    "Okay, can you submit the purchase order",
    ]

print("Welcome to the Inventory Reordering Supervisor Agent!")
state = {"messages": []}

    #while True:
for question in questions:
        #user_input = input("\nUser > ")
        user_input = question
        print('User Question: ', user_input)
        if user_input.lower() in ["quit", "exit"]: break
        
        state["messages"].append(HumanMessage(content=user_input))
        
        final_state = reorder_agent.invoke(state)
        
        final_response_message = final_state['messages'][-1]
        print(f"\nAgent > {final_response_message.content}")

        state['messages'].extend(final_state['messages'])

Welcome to the Inventory Reordering Supervisor Agent!
User Question:  Can you generate a purchase order for me? Budget is $1400.
---[Supervisor]: Processing user request---
---[Supervisor]: Executing tool---
 -> Calling tool: generate_purchase_order_recommendation

Agent > You need to select a valid strategy to generate a recommendation. Here are your options:
- **Balanced Profit**: A well-rounded approach prioritizing profitability while maintaining good service levels and managing costs.
- **Peak Season Prep**: Aggressively stocks up to prevent stockouts during high-demand periods, prioritizing service level and speed over cost.
- **Cash Preservation**: Minimizes immediate spending by ordering only the most essential items, focusing on low costs and preserving capital.
- **Service Excellence**: Focuses on maximizing customer satisfaction by maintaining high stock levels for all items, ensuring availability.

Please choose one, or you can ask me to view the settings, edit a strategy, 

In [12]:
# reporting / sql questions
questions = [
        "can i see all inventory under the reorder threshold in a table", 
        "what are the supplier details of skus with the longest lead times of items with the highest sales in terms of quantity",
        "i want to get the quantity of items sold based on the product category sort in descending order",
        # "can you show me details of the top 5 customers with the highest return rate?",
        # "Who is our top customer by sales volume?",
        # "Show me the top 5 most sold SKUs along with the sku name?",
        # "what is the supplier name i most buy from?",
        # "what is the gender distribution among our customers"

    ]

for question_text in questions:

        inputs = {"messages": [HumanMessage(content=question_text)], "retry_count": 0}
        
        print(f"\n\n RUNNING AGENT FOR: '{question_text}")
        
        final_state = None
        for event in sql_agent.stream(inputs, {"recursion_limit": 10}):
            for key, value in event.items():
                print(f"--- (Event from Node: {key}) ---")
            final_state = list(event.values())[0]

        print(f"\n\n{'='*30}\nFINAL RESPONSE\n{'='*30}\n")
        if final_state and final_state.get("messages"):
            final_response_message = final_state["messages"][-1]
            print(final_response_message.content)



 RUNNING AGENT FOR: 'can i see all inventory under the reorder threshold in a table
---NODE: GENERATE SQL---
 -> Generated SQL: SELECT * FROM products WHERE current_stock <= reorder_threshold
--- (Event from Node: generate_sql) ---
---NODE: EXECUTE SQL---
Full query string received: SELECT * FROM products WHERE current_stock <= reorder_threshold
 -> Executing statement 1/1: SELECT * FROM products WHERE current_stock <= reorder_threshold
 -> DB Results: [('SKU0002', 'Glass Guess', 'Grocery', 48, 142, 45.69, 56.51, 'STORE03', 'Relationship wind ball travel then coach.', 71, None), ('SKU0010', 'Turn I', 'Home', 49, 198, 7.02, 88.26, 'STORE04', 'Key tree body player bag beat.', 68, None), ('SKU0012', 'Cover Letter', 'Books', 30, 154, 41.12, 101.9, 'STORE07', 'Act entire western program.', 51, 'New Arrivals'), ('SKU0021', 'Often Theory', 'Home', 9, 188, 47.06, 106.37, 'STORE01', 'Parent hundred partner bank into.', 28, None), ('SKU0023', 'Democratic Eat', 'Home', 11, 198, 38.28, 89.15, 'S

In [12]:
import os
import sqlite3
import pandas as pd

conn = sqlite3.connect(os.getenv("DB_PATH"))


products_df = pd.read_sql_query("SELECT T1.customer_id, T1.name, SUM(T2.amount) AS total_sales FROM customers AS T1 INNER JOIN sales AS T2 ON T1.customer_id = T2.customer_id GROUP BY T1.customer_id ORDER BY total_sales DESC LIMIT 1", conn)
products_df

,customer_id,name,total_sales
0,C0141,Theresa Woods,7401.64


In [ ]:
from agents.data_retriever import DataRetriever
from agents.reorder_agent import create_and_solve_reordering_model,view_purchase_order_recommendations
import os
import sqlite3
DB_PATH = os.getenv("DB_PATH")

REORDERING_STRATEGY_PRESETS = {
    'Balanced Profit': {'priorities': {'profit': 8, 'service_level': 6, 'cost_minimization': 5, 'delivery_speed': 3}},
    'Peak Season Prep': {'priorities': {'service_level': 10, 'delivery_speed': 9, 'profit': 3, 'cost_minimization': 2}},
    'Cash Preservation': {'priorities': {'cost_minimization': 10, 'profit': 4, 'service_level': 2, 'delivery_speed': 1}},
    'Service Excellence': {'priorities': {'service_level': 10, 'profit': 8, 'delivery_speed': 6, 'cost_minimization': 3}}
}

strategy=list(REORDERING_STRATEGY_PRESETS.keys())[2]


plan = create_and_solve_reordering_model(DataRetriever(DB_PATH),REORDERING_STRATEGY_PRESETS[strategy],1300)

print(plan)
recommended_po = plan.get('reordering_plan', [])

try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("DELETE FROM draft_purchase_order;") # Clear previous draft
        for item in recommended_po:
            cost_per_unit = item['cost'] / item['quantity_to_order'] if item.get('quantity_to_order') else 0
            cursor.execute("INSERT INTO draft_purchase_order (sku_id, supplier_id, quantity_to_order, cost_per_unit, lead_time) VALUES (?, ?, ?, ?, ?)",
                           (item['sku_id'], item['supplier'], item['quantity_to_order'], cost_per_unit, item['lead_time']))
            
            
        conn.commit()

except Exception as e:
         print(f"Database error: {e}")
finally:
        if conn: conn.close()

markdown_table = view_purchase_order_recommendations()
print(markdown_table)


Generating a realistic recommendation...
Fetching supplier SKU data...
Fetching product data...
Fetching forecast data...
Fetching product data...

Calculating Target Inventory Levels (Forecast + Safety Stock)...
{'status': 'Optimal', 'priorities': {'priorities': {'cost_minimization': 10, 'profit': 4, 'service_level': 2, 'delivery_speed': 1}}, 'objective_value': 820315.7000000002, 'total_cost': 1294.31, 'reordering_plan': [{'sku_id': 'SKU0012', 'supplier': 'S018', 'quantity_to_order': 10, 'cost': 207.39999999999998, 'lead_time': 2}, {'sku_id': 'SKU0021', 'supplier': 'S010', 'quantity_to_order': 19, 'cost': 353.40000000000003, 'lead_time': 4}, {'sku_id': 'SKU0023', 'supplier': 'S004', 'quantity_to_order': 6, 'cost': 95.76, 'lead_time': 10}, {'sku_id': 'SKU0027', 'supplier': 'S003', 'quantity_to_order': 6, 'cost': 68.03999999999999, 'lead_time': 2}, {'sku_id': 'SKU0064', 'supplier': 'S019', 'quantity_to_order': 1, 'cost': 16.29, 'lead_time': 3}, {'sku_id': 'SKU0067', 'supplier': 'S017', 

In [5]:

import os
import sqlite3
import pandas as pd

conn = sqlite3.connect(os.getenv("DB_PATH"))


products_df = pd.read_sql_query("SELECT sku_id, product_name, product_category, category, current_stock, reorder_threshold,max_capacity FROM products", conn)

demand_forecast_df = pd.read_sql_query("SELECT sku_id, quantity as demand FROM demand_forecast", conn)

df = products_df.merge(demand_forecast_df, on = "sku_id",how='left')
df['pred_stock'] = df['current_stock']-df['demand']
df.sort_values(['pred_stock'],ascending=True).head(30)


,sku_id,product_name,product_category,category,current_stock,reorder_threshold,max_capacity,demand,pred_stock
20,SKU0021,Often Theory,Home,None,9,28,188,28,-19
80,SKU0081,Happen Mr,Electronics,New Arrivals,27,20,46,38,-11
66,SKU0067,Just Yard,Clothing,None,17,32,109,25,-8
97,SKU0098,Choice Environmental,Electronics,None,25,63,172,31,-6
86,SKU0087,Language Simply,Home,Seasonal Items,9,16,43,13,-4
22,SKU0023,Democratic Eat,Home,New Arrivals,11,88,198,14,-3
26,SKU0027,Community Set,Home,New Arrivals,14,47,150,16,-2
11,SKU0012,Cover Letter,Books,New Arrivals,30,51,154,32,-2
13,SKU0014,Attention True,Books,None,19,3,19,14,5
63,SKU0064,Show Share,Home,New Arrivals,27,34,159,22,5


In [9]:
df.sort_values(['demand'],ascending=False)

,sku_id,product_name,product_category,category,current_stock,reorder_threshold,max_capacity,demand
38,SKU0039,Let Knowledge,Clothing,Seasonal Items,85,74,186,52
74,SKU0075,Image Work,Home,None,161,49,168,49
80,SKU0081,Happen Mr,Electronics,New Arrivals,27,20,46,38
43,SKU0044,Meet Again,Clothing,None,107,68,154,34
54,SKU0055,Federal How,Grocery,None,187,65,194,33
...,...,...,...,...,...,...,...,...
60,SKU0061,Whom Customer,Grocery,New Arrivals,82,19,82,3
56,SKU0057,Medical Away,Books,None,154,43,154,3
44,SKU0045,Sound I,Home,Stagnant Items,60,14,60,2
61,SKU0062,Above Add,Electronics,None,198,97,198,1
